In [1]:
# WARNING: you are on the master branch; please refer to examples on the branch corresponding to your `cortex version` (e.g. for version 0.18.*, run `git checkout -b 0.18` or switch to the `0.18` branch on GitHub)

import torch
import regex
import tqdm
import re


roberta = torch.hub.load("pytorch/fairseq", "roberta.large")
roberta.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"using device: {device}")
roberta.to(device)

model = roberta
en_stopwords = ["a","about","above","across","after","again","against","all","almost","alone","along","already","also","although","always","among","an","and","another","any","anybody","anyone","anything","anywhere","are","area","areas","around","as","ask","asked","asking","asks","at","away","b","back","backed","backing","backs","be","became","because","become","becomes","been","before","began","behind","being","beings","best","better","between","big","both","but","by","c","came","can","cannot","case","cases","certain","certainly","clear","clearly","come","could","d","did","differ","different","differently","do","does","done","down","down","downed","downing","downs","during","e","each","early","either","end","ended","ending","ends","enough","even","evenly","ever","every","everybody","everyone","everything","everywhere","f","face","faces","fact","facts","far","felt","few","find","finds","first","for","four","from","full","fully","further","furthered","furthering","furthers","g","gave","general","generally","get","gets","give","given","gives","go","going","good","goods","got","great","greater","greatest","group","grouped","grouping","groups","h","had","has","have","having","he","her","here","herself","high","high","high","higher","highest","him","himself","his","how","however","i","if","important","in","interest","interested","interesting","interests","into","is","it","its","itself","j","just","k","keep","keeps","kind","knew","know","known","knows","l","large","largely","last","later","latest","least","less","let","lets","like","likely","long","longer","longest","m","made","make","making","man","many","may","me","member","members","men","might","more","most","mostly","mr","mrs","much","must","my","myself","n","necessary","need","needed","needing","needs","never","new","new","newer","newest","next","no","nobody","non","noone","not","nothing","now","nowhere","number","numbers","o","of","off","often","old","older","oldest","on","once","one","only","open","opened","opening","opens","or","order","ordered","ordering","orders","other","others","our","out","over","p","part","parted","parting","parts","per","perhaps","place","places","point","pointed","pointing","points","possible","present","presented","presenting","presents","problem","problems","put","puts","q","quite","r","rather","really","right","right","room","rooms","s","said","same","saw","say","says","second","seconds","see","seem","seemed","seeming","seems","sees","several","shall","she","should","show","showed","showing","shows","side","sides","since","small","smaller","smallest","so","some","somebody","someone","something","somewhere","state","states","still","still","such","sure","t","take","taken","than","that","the","their","them","then","there","therefore","these","they","thing","things","think","thinks","this","those","though","thought","thoughts","three","through","thus","to","today","together","too","took","toward","turn","turned","turning","turns","two","u","under","until","up","upon","us","use","used","uses","v","very","w","want","wanted","wanting","wants","was","way","ways","we","well","wells","went","were","what","when","where","whether","which","while","who","whole","whose","why","will","with","within","without","work","worked","working","works","would","x","y","year","years","yet","you","young","younger","youngest","your","yours","z"]
en_banned_words = ['Hitler', 'hitler']


Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /Users/zhengyuan.ma/.cache/torch/hub/master.zip


Unable to build Cython components. Please make sure Cython is installed if the torch.hub model you are loading depends on it.


100%|██████████| 655283069/655283069 [00:25<00:00, 25360588.56B/s]
1042301B [00:00, 2677686.32B/s]
456318B [00:00, 2033320.81B/s]

using device: cpu


In [29]:
# while 
payload= {'text': 'Donald Trump is'}
predictions = model.fill_mask(payload["text"] + " <mask>", topk=20)
sorted([post_process(prediction) for prediction in predictions if post_process(prediction)], key= lambda x:-x[1])[:10]

[(' president', 0.028827011585235596),
 (' President', 0.02342144586145878),
 (' gone', 0.013789468444883823),
 (' a', 0.012962009012699128),
 (' racist', 0.011698862537741661),
 (' Trump', 0.010534184984862804),
 (' wrong', 0.010439056903123856),
 (' dead', 0.009124334901571274),
 (' elected', 0.008852891623973846),
 (' Jewish', 0.008699529804289341)]

In [28]:
def post_process(prediction):
    if not prediction[-1].strip().isalpha():
        return None
    if prediction[-1].strip() in en_banned_words:
        return None
    if prediction[-1].strip() in en_stopwords:
        return (prediction[2], prediction[1] * 0.4)
    return (prediction[2], prediction[1])